# Install Dependencies

In [ ]:
%%shell

# Clone PromptSRC code base
git clone https://github.com/Arshia-HZ/Deep-Learning-Project.git

cd Deep-Learning-Project/
# Install requirements

pip install -r requirements.txt -q

# Update setuptools package
# pip install setuptools==59.5.0 -q  # Keep this commented or remove it

cd Dassl.pytorch/

# Install dependencies
pip install -r requirements.txt -q

# Upgrade setuptools to a version compatible with Python 3.12
pip install --upgrade setuptools -q

# Create pyproject.toml to enable modern editable install
cat <<EOF > pyproject.toml
[build-system]
requires = ["setuptools>=61.0", "wheel"]
build-backend = "setuptools.build_meta"
EOF

# Install this library in editable mode without build isolation
pip install --no-build-isolation -e .

Cloning into 'Deep-Learning-Project'...
remote: Enumerating objects: 756, done.
remote: Counting objects: 100% (756/756), done.
remote: Compressing objects: 100% (533/533), done.
remote: Total 756 (delta 244), reused 696 (delta 196), pack-reused 0 (from 0)
Receiving objects: 100% (756/756), 9.44 MiB | 15.62 MiB/s, done.
Resolving deltas: 100% (244/244), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.3/185.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
%cd Deep-Learning-Project/

/content/Deep-Learning-Project


# Prepare Dataset

In [ ]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("wenewone/cub2002011")

print("Path to dataset files:", path)

# Create the destination directory if it doesn't exist
os.makedirs("/content/data", exist_ok=True)

# Construct the path to the CUB_200_2011 directory within the downloaded content
cub_path = os.path.join(path, "CUB_200_2011")

# Move the CUB_200_2011 directory to /content/data
!mv -v "{cub_path}" /content/data/

Streaming output truncated to the last 5000 lines.
mv: cannot remove '/kaggle/input/cub2002011/CUB_200_2011/images/058.Pigeon_Guillemot/Pigeon_Guillemot_0040_40270.jpg': Read-only file system
mv: cannot remove '/kaggle/input/cub2002011/CUB_200_2011/images/058.Pigeon_Guillemot/Pigeon_Guillemot_0014_40040.jpg': Read-only file system
mv: cannot remove '/kaggle/input/cub2002011/CUB_200_2011/images/058.Pigeon_Guillemot/Pigeon_Guillemot_0026_40126.jpg': Read-only file system
mv: cannot remove '/kaggle/input/cub2002011/CUB_200_2011/images/058.Pigeon_Guillemot/Pigeon_Guillemot_0077_39885.jpg': Read-only file system
mv: cannot remove '/kaggle/input/cub2002011/CUB_200_2011/images/058.Pigeon_Guillemot/Pigeon_Guillemot_0057_40130.jpg': Read-only file system
mv: cannot remove '/kaggle/input/cub2002011/CUB_200_2011/images/058.Pigeon_Guillemot/Pigeon_Guillemot_0025_40511.jpg': Read-only file system
mv: cannot remove '/kaggle/input/cub2002011/CUB_200_2011/images/058.Pigeon_Guillemot/Pigeon_Guillemot_0

In [ ]:
import os
import json
from collections import defaultdict
from sklearn.model_selection import train_test_split

# === CONFIG ===
cub_root = "/content/data/CUB_200_2011"  # Updated path to extracted CUB_200_2011 directory
images_txt = os.path.join(cub_root, "images.txt")
labels_txt = os.path.join(cub_root, "image_class_labels.txt")
split_txt  = os.path.join(cub_root, "train_test_split.txt")
out_json = "/content/data/CUB_200_2011/cub_200_2011_splits.json"
val_frac = 0.1          # fraction of training set to use as validation (e.g. 0.1 => 10%)
zero_based = True      # if True, outputs class indices zero-based; else keeps original (1-based)
# ==============

def read_map(path, sep=None, names=2):
    """
    Read two-column text files where columns are separated by whitespace.
    Returns dict[int]->str for first->second column.
    """
    d = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < names:
                continue
            key = int(parts[0])
            # join the rest for values that contain spaces (shouldn't in CUB, but safe)
            val = " ".join(parts[1:])
            d[key] = val
    return d


assert os.path.isdir(cub_root), f"Could not find directory '{cub_root}'. Extract CUB_200_2011 there."

# id -> relative image path (as in images.txt)
images = read_map(images_txt, names=2)
# id -> class_id (integer)
image_class_labels = {k: int(v) for k, v in read_map(labels_txt, names=2).items()}
# id -> is_training_img (1 or 0)
# Renamed the variable to avoid conflict with the imported function
train_test_split_data = {k: int(v) for k, v in read_map(split_txt, names=2).items()}

# build list of samples
samples = []
for img_id, rel_path in images.items():
    cls_id = image_class_labels.get(img_id)
    if cls_id is None:
        continue
    # Use the renamed variable here
    is_train = train_test_split_data.get(img_id, 0) == 1
    # Use rel_path directly
    rel = rel_path.replace("\\", "/")
    # class_name: folder name before first slash (e.g. "001.Black_footed_Albatross")
    class_folder = rel_path.split("/", 1)[0] if "/" in rel_path else rel_path
    # optionally strip the numeric prefix (e.g. "001.") and keep only readable name
    if "." in class_folder:
        class_name = class_folder.split(".", 1)[1]
    else:
        class_name = class_folder

    idx = cls_id - 1 if zero_based else cls_id

    samples.append({
        "img_id": img_id,
        "path": rel, # Use the relative path without "images/" prefix
        "class_id": idx,
        "class_name": class_name,
        "is_train": is_train
    })

# separate train / test using the provided split
train_samples = [s for s in samples if s["is_train"]]
test_samples  = [s for s in samples if not s["is_train"]]

# create a validation split from the training set (stratified by class)
train_paths = [s["path"] for s in train_samples]
train_labels = [s["class_id"] for s in train_samples]
if val_frac > 0:
    # Use the imported function train_test_split here
    tr_paths, val_paths, tr_idx, val_idx = train_test_split(
        train_paths, list(range(len(train_paths))),
        test_size=val_frac, random_state=42, stratify=train_labels
    )
    # build lists
    new_train = []
    val_list = []
    for i in tr_idx:
        s = train_samples[i]
        new_train.append([s["path"], s["class_id"], s["class_name"]])
    for i in val_idx:
        s = train_samples[i]
        val_list.append([s["path"], s["class_id"], s["class_name"]])
else:
    new_train = [[s["path"], s["class_id"], s["class_name"]] for s in train_samples]
    val_list = []

test_list = [[s["path"], s["class_id"], s["class_name"]] for s in test_samples]

out = {"train": new_train, "val": val_list, "test": test_list}
with open(out_json, "w", encoding="utf-8") as f:
    json.dump(out, f, indent=2, ensure_ascii=False)

print(f"Saved JSON with splits -> {out_json}")
print(f"Train: {len(new_train)}, Val: {len(val_list)}, Test: {len(test_list)}")

Saved JSON with splits -> /content/data/CUB_200_2011/cub_200_2011_splits.json
Train: 5394, Val: 600, Test: 5794


# Base-to-novel

## Maple

In [ ]:
# trains and evaluates on base classes
!bash scripts/maple/base2new_train_maple.sh CUB200 1
# evaluates on novel classes
!bash scripts/maple/base2new_test_maple.sh CUB200 1

# seed=3
# trains and evaluates on base classes
!bash scripts/maple/base2new_train_maple.sh CUB200 3
# evaluates on novel classes
!bash scripts/maple/base2new_test_maple.sh CUB200 3

# seed=5
# trains and evaluates on base classes
!bash scripts/maple/base2new_train_maple.sh CUB200 5
# evaluates on novel classes
!bash scripts/maple/base2new_test_maple.sh CUB200 5

Run this job and save the output to output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed1
2025-08-21 18:41:18.069374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755801678.089495    1966 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755801678.095553    1966 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755801678.110958    1966 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755801678.110989    1966 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking t

In [ ]:
# prints averaged results for base classes
!python parse_test_res.py /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx
# averaged results for novel classes
!python parse_test_res.py /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx --test-log

Parsing files in /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed1/log.txt. accuracy: 77.23%. 
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed3/log.txt. accuracy: 76.43%. 
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed5/log.txt. accuracy: 75.73%. 
===
Summary of directory: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx
* accuracy: 76.46% +- 0.61%
===
Parsing files in /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx
file: /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed1/log.txt. accuracy: 52.90%. 
fil

## Co-CoOp

In [ ]:
# seed=1
!bash scripts/cocoop/base2new_train.sh CUB200 1
!bash scripts/cocoop/base2new_test.sh CUB200 1

# seed=2
!bash scripts/cocoop/base2new_train.sh CUB200 2
!bash scripts/cocoop/base2new_test.sh CUB200 2

# seed=3
!bash scripts/cocoop/base2new_train.sh CUB200 3
!bash scripts/cocoop/base2new_test.sh CUB200 3

Run this job and save the output to output/base2new/train_base/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1/seed1
2025-08-20 19:50:01.260530: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755719401.281969   10098 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755719401.288527   10098 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755719401.304506   10098 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755719401.304531   10098 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
# prints averaged results for base classes
!python parse_test_res.py /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1
# averaged results for novel classes
!python parse_test_res.py /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1 --test-log

Parsing files in /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1/seed1/log.txt. accuracy: 72.77%. 
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1/seed2/log.txt. accuracy: 72.91%. 
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1/seed3/log.txt. accuracy: 72.35%. 
===
Summary of directory: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1
* accuracy: 72.68% +- 0.24%
===
Parsing files in /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1
file: /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1/seed1/log.txt. a

## Protext

In [ ]:
# trains on base classes and then evaluate on both base and novel classes
!bash scripts/protext/base2novel.sh CUB200 output/experiment/path

2025-08-20 18:05:26.508517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755713126.528225     837 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755713126.534239     837 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755713126.549584     837 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755713126.549609     837 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755713126.549615     837 computation_placer.cc:177] computation placer alr

## CLIP

In [ ]:
!bash scripts/zsclip_cupl/zeroshot.sh CUB200 base None output/experiment/clip_zeroshot/base

2025-08-20 18:54:02.401278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755716042.420654    2313 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755716042.426706    2313 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755716042.441764    2313 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755716042.441792    2313 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755716042.441796    2313 computation_placer.cc:177] computation placer alr

In [ ]:
!bash scripts/zsclip_cupl/zeroshot.sh CUB200 new None output/experiment/clip_zeroshot/novel

2025-08-20 18:54:40.992726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755716081.012843    2547 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755716081.018891    2547 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755716081.033968    2547 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755716081.033999    2547 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755716081.034003    2547 computation_placer.cc:177] computation placer alr

## CuPL

In [ ]:
!bash scripts/zsclip_cupl/zeroshot.sh CUB200 base  /content/Deep-Learning-Project/templates/CUB200_prompts_full.json output/experiment/cupl/

2025-08-20 20:57:01.654280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755723421.685541    3783 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755723421.694732    3783 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755723421.709859    3783 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755723421.709888    3783 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755723421.709891    3783 computation_placer.cc:177] computation placer alr

In [ ]:
!bash scripts/zsclip_cupl/zeroshot.sh CUB200 new  /content/Deep-Learning-Project/templates/CUB200_prompts_full.json output/experiment/cupl/

2025-08-20 20:57:44.187608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755723464.217834    4045 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755723464.223925    4045 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755723464.239091    4045 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755723464.239117    4045 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755723464.239121    4045 computation_placer.cc:177] computation placer alr

## MaPLe Initialized ProText

In [ ]:
# trains and evaluates on base classes
!bash scripts/maple/base2new_train_maple.sh CUB200 1
# evaluates on novel classes
!bash scripts/maple/base2new_test_maple.sh CUB200 1

# seed=3
# trains and evaluates on base classes
!bash scripts/maple/base2new_train_maple.sh CUB200 3
# evaluates on novel classes
!bash scripts/maple/base2new_test_maple.sh CUB200 3

# seed=5
# trains and evaluates on base classes
!bash scripts/maple/base2new_train_maple.sh CUB200 5
# evaluates on novel classes
!bash scripts/maple/base2new_test_maple.sh CUB200 5

Run this job and save the output to output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed1
2025-08-21 18:23:28.564162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755800608.596906   20446 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755800608.606768   20446 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755800608.632015   20446 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755800608.632059   20446 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking t

In [ ]:
# prints averaged results for base classes
!python parse_test_res.py /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx
# averaged results for novel classes
!python parse_test_res.py /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx --test-log

Parsing files in /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed1/log.txt. accuracy: 77.41%. 
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed3/log.txt. accuracy: 76.96%. 
file: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed5/log.txt. accuracy: 76.82%. 
===
Summary of directory: /content/Deep-Learning-Project/output/base2new/train_base/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx
* accuracy: 77.06% +- 0.25%
===
Parsing files in /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx
file: /content/Deep-Learning-Project/output/base2new/test_new/CUB200/shots_16/MaPLe/vit_b16_c2_ep5_batch4_2ctx/seed1/log.txt. accuracy: 52.22%. 
fil

# Cross-Dataset

## Train TinyImagenet

In [ ]:
import os
import shutil
import subprocess

# --- 1. Configuration ---
# We'll set up the data in Colab's temporary storage
DATA_ROOT = '/content/data'
# os.environ['DATA'] = DATA_ROOT # Set environment variable for your scripts
TARGET_IMAGENET_DIR = os.path.join(DATA_ROOT, 'imagenet')
TINY_IMAGENET_ZIP = 'tiny-imagenet-200.zip'
TINY_IMAGENET_DIR = 'tiny-imagenet-200'

print("🚀 Starting Tiny ImageNet setup for Google Colab...")

# --- 2. Download and Unzip ---
print("\nDownloading Tiny ImageNet (approx. 240 MB)...")
if not os.path.exists(TINY_IMAGENET_ZIP):
    subprocess.run(['wget', '-q', 'https://image-net.org/data/tiny-imagenet-200.zip'])
else:
    print("Zip file already downloaded.")

print("Unzipping data...")
if os.path.exists(TINY_IMAGENET_DIR):
    shutil.rmtree(TINY_IMAGENET_DIR) # Clean up previous extraction
subprocess.run(['unzip', '-q', TINY_IMAGENET_ZIP])

# --- 3. Create Target Directory Structure ---
print(f"\nCreating target structure in {TARGET_IMAGENET_DIR}...")
TARGET_IMAGES_DIR = os.path.join(TARGET_IMAGENET_DIR, 'images')
TARGET_TRAIN_DIR = os.path.join(TARGET_IMAGES_DIR, 'train')
TARGET_VAL_DIR = os.path.join(TARGET_IMAGES_DIR, 'val')

# Clean up previous runs if they exist
if os.path.exists(TARGET_IMAGENET_DIR):
    shutil.rmtree(TARGET_IMAGENET_DIR)

os.makedirs(TARGET_TRAIN_DIR)
os.makedirs(TARGET_VAL_DIR)

# --- 4. Restructure the Validation Set ---
print("Processing validation set...")
val_annotations_file = os.path.join(TINY_IMAGENET_DIR, 'val', 'val_annotations.txt')
val_images_dir = os.path.join(TINY_IMAGENET_DIR, 'val', 'images')

with open(val_annotations_file, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        image_name, class_id = parts[0], parts[1]

        class_dir = os.path.join(TARGET_VAL_DIR, class_id)
        os.makedirs(class_dir, exist_ok=True)

        src_path = os.path.join(val_images_dir, image_name)
        if os.path.exists(src_path):
            shutil.copy(src_path, class_dir)
print("Validation set restructured.")

# --- 5. Restructure the Training Set ---
print("Processing training set...")
original_train_dir = os.path.join(TINY_IMAGENET_DIR, 'train')
for class_id in os.listdir(original_train_dir):
    # Move images from the 'images' subdirectory to the class directory
    original_class_dir = os.path.join(original_train_dir, class_id, 'images')
    target_class_dir = os.path.join(TARGET_TRAIN_DIR, class_id)
    if os.path.isdir(original_class_dir):
        shutil.copytree(original_class_dir, target_class_dir)
print("Training set restructured.")

# --- 6. Create the classnames.txt File ---
print("Creating classnames.txt...")
words_file = os.path.join(TINY_IMAGENET_DIR, 'words.txt')
wnids_file = os.path.join(TINY_IMAGENET_DIR, 'wnids.txt')
classnames_file = os.path.join(TARGET_IMAGENET_DIR, 'classnames.txt')

# First, get the set of 200 class IDs that are actually in the dataset
with open(wnids_file, 'r') as f:
    class_ids_in_use = {line.strip() for line in f}


with open(words_file, 'r') as infile, open(classnames_file, 'w') as outfile:
    for line in infile:
        if not line.strip(): continue # Skip empty lines

        class_id = line.strip().split('\t')[0]

        # Only write the entry if its ID is in our set of 200
        if class_id in class_ids_in_use:
            full_name = line.strip().split('\t')[1]
            simple_name = full_name.split(',')[0].strip()
            outfile.write(f"{class_id} {simple_name}\n")
print("classnames.txt created.")

# --- 7. Final Cleanup ---
print("Cleaning up temporary files...")
shutil.rmtree(TINY_IMAGENET_DIR)
# To save space, you can also remove the zip file by uncommenting the next line
# os.remove(TINY_IMAGENET_ZIP)

print("\n✅ All done! Your Tiny ImageNet dataset is ready at:")
print(f"'{TARGET_IMAGENET_DIR}'")

🚀 Starting Tiny ImageNet setup for Google Colab...

Unzipping data...

Creating target structure in /content/data/imagenet...
Processing validation set...
Validation set restructured.
Processing training set...
Training set restructured.
Creating classnames.txt...
classnames.txt created.
Cleaning up temporary files...

✅ All done! Your Tiny ImageNet dataset is ready at:
'/content/data/imagenet'


## Maple

In [ ]:
%%shell

# seed=1
bash scripts/maple/xd_train_maple.sh imagenet 1
# seed=3
bash scripts/maple/xd_train_maple.sh imagenet 3
# seed=5
bash scripts/maple/xd_train_maple.sh imagenet 5

Run this job and save the output to output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed1
2025-08-21 20:58:54.203204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755809934.225163     960 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755809934.231879     960 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755809934.247334     960 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755809934.247359     960 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the s

In [ ]:
# prints averaged results for imagenet result train
!python parse_test_res.py /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots

Parsing files in /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots
file: /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed1/log.txt. accuracy: 73.13%. 
file: /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed3/log.txt. accuracy: 73.15%. 
file: /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed5/log.txt. accuracy: 73.65%. 
===
Summary of directory: /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots
* accuracy: 73.31% +- 0.24%
===


In [ ]:
%%shell

for SEED in 1 3 5
do
    bash scripts/maple/xd_test_maple.sh CUB200 ${SEED}
done

Run this job and save the output to output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200/seed1
2025-08-21 22:11:55.986396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755814316.006775   20240 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755814316.012866   20240 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755814316.028421   20240 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755814316.028456   20240 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid link

In [ ]:
# averaged results for CUB200 result test
!python parse_test_res.py /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200 --test-log

Parsing files in /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200
file: /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200/seed1/log.txt. accuracy: 52.80%. 
file: /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200/seed3/log.txt. accuracy: 50.76%. 
file: /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200/seed5/log.txt. accuracy: 50.97%. 
===
Summary of directory: /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200
* accuracy: 51.51% +- 0.92%
===


## Co-CoOp

In [ ]:
%%shell

# seed=1
bash scripts/cocoop/xd_train.sh 1

# seed=2
bash scripts/cocoop/xd_train.sh 2

Run this job and save the output to output/imagenet/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1_16shots/seed1
2025-08-21 04:14:23.669344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755749663.689065    1254 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755749663.695156    1254 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755749663.710515    1254 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755749663.710541    1254 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target m

In [ ]:
%%shell

for SEED in 1 2
do
    bash scripts/cocoop/xd_test.sh CUB200 ${SEED}
done

Run this job and save the output to output/evaluation/CoCoOp/vit_b16_c4_ep10_batch1_ctxv1_16shots/CUB200/seed1
2025-08-21 07:55:23.519676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755762923.542194   56326 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755762923.549354   56326 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755762923.566322   56326 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755762923.566347   56326 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same

## Protext

In [ ]:
%%shell

# The second argument is the path for saving logs and model weights
bash scripts/protext/cross_datasets_train.sh imagenet output/imagenet_cross_dataset

2025-08-21 10:41:03.041618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755772863.081858    2153 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755772863.091936    2153 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755772863.115600    2153 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755772863.115636    2153 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755772863.115643    2153 computation_placer.cc:177] computation placer alr

In [ ]:
%%shell

# Dassl.pytorch\dassl\data\data_manager.py line 103 change into False
# Other possible dataset values includes [imagenet, food101, dtd, ucf101, oxford_flowers, fgvc_aircraft, sun397, eurosat]
# The second argument is the folder path for the pretrained model weight
bash scripts/protext/cross_datasets_test.sh CUB200 output/imagenet_cross_dataset

2025-08-21 11:15:50.343504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755774950.363830   11413 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755774950.370028   11413 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755774950.385914   11413 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755774950.385941   11413 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755774950.385947   11413 computation_placer.cc:177] computation placer alr

## CLIP

In [ ]:
!bash scripts/zsclip_cupl/zeroshot.sh CUB200 all None output/experiment/clip_zeroshot/cross

2025-08-20 22:56:45.543423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755730605.564207    5873 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755730605.570792    5873 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755730605.586205    5873 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755730605.586231    5873 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755730605.586235    5873 computation_placer.cc:177] computation placer alr

## CuPL

In [ ]:
!bash scripts/zsclip_cupl/zeroshot.sh CUB200 all /content/Deep-Learning-Project/templates/CUB200_prompts_full.json output/experiment/cupl/cross

2025-08-20 22:58:54.787476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755730734.809282    6518 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755730734.815537    6518 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755730734.831116    6518 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755730734.831142    6518 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755730734.831145    6518 computation_placer.cc:177] computation placer alr

## MaPLe Initialized ProText

In [ ]:
%%shell

# seed=1
bash scripts/maple/xd_train_maple.sh imagenet 1
# seed=3
bash scripts/maple/xd_train_maple.sh imagenet 3
# seed=5
bash scripts/maple/xd_train_maple.sh imagenet 5

Run this job and save the output to output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed1
2025-08-21 20:40:57.661282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755808857.685780   58126 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755808857.692042   58126 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755808857.712016   58126 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755808857.712056   58126 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the s

In [ ]:
%%shell

for SEED in 1 3 5
do
    bash scripts/maple/xd_test_maple.sh CUB200 ${SEED}
done

Run this job and save the output to output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200/seed1
2025-08-21 21:51:10.570195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755813070.591170   76464 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755813070.597719   76464 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755813070.613611   76464 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755813070.613639   76464 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid link

In [ ]:
# prints averaged results for base classes
!python parse_test_res.py /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots
# averaged results for novel classes

Parsing files in /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots
file: /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed1/log.txt. accuracy: 73.39%. 
file: /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed3/log.txt. accuracy: 73.24%. 
file: /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/seed5/log.txt. accuracy: 74.04%. 
===
Summary of directory: /content/Deep-Learning-Project/output/imagenet/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots
* accuracy: 73.56% +- 0.35%
===


In [ ]:
!python parse_test_res.py /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200 --test-log

Parsing files in /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200
file: /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200/seed1/log.txt. accuracy: 51.12%. 
file: /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200/seed3/log.txt. accuracy: 51.40%. 
file: /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200/seed5/log.txt. accuracy: 52.12%. 
===
Summary of directory: /content/Deep-Learning-Project/output/evaluation/MaPLe/vit_b16_c2_ep5_batch4_2ctx_cross_datasets_16shots/CUB200
* accuracy: 51.55% +- 0.42%
===
